In [1]:
# Parameters
TEST = True
QUICK_RUN = True


In [2]:
import numpy
import pandas as pd
import os
import sys

In [3]:
sys.path.insert(0,"/usr/local/lib/python3.9/site-packages")

In [4]:
import pyspark
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
import pymysql

In [5]:
host_name = os.environ.get('host_name')
user = "group2"
passwd = os.environ.get('aws_password')
db="group2db"
# connect to MySQL
con = pymysql.connect(host=host_name,user=user,passwd=passwd,db=db)
cursor = con.cursor()
SQL_Query = pd.read_sql_query(
'''select * from tweets_with_location ''', con)

In [6]:
df = pd.DataFrame(SQL_Query)

In [7]:
spark = SparkSession \
    .builder \
    .appName("data_import") \
    .config("spark.dynamicAllocation.enabled", "true") \
    .config("spark.shuffle.service.enabled", "true") \
    .enableHiveSupport() \
    .getOrCreate()

In [8]:
df_sp = spark.createDataFrame(df)

In [9]:
df_sp.show()

+--------+---------------+-------------------+--------------------+-------------+-----------+-----+
|tweet_id|       username|         created_at|               tweet|retweet_count|   location|place|
+--------+---------------+-------------------+--------------------+-------------+-----------+-----+
|       1|    Not_Silvers|2021-01-11 21:32:20|RT @tamrieldude: ...|            0|       Ohio| null|
|       2|  EverydayEagle|2021-01-11 21:33:03|RT @GrassRootRevo...|            0|      Texas| null|
|       3|        LaRene1|2021-01-11 21:33:22|@akgraham7 @Klapt...|            0|    Florida| null|
|       4|       Crankzie|2021-01-11 21:33:48|@ZitoFN_ LMAO THI...|            0| California| null|
|       5|       senschuh|2021-01-11 21:34:06|RT @jbryan522: Ma...|            0|Mississippi| null|
|       6|RachelleBlidner|2021-01-11 21:34:23|500 people showed...|            0|   New York| null|
|       7|       puffypcb|2021-01-11 21:34:29|@GovRonDeSantis H...|            0|    Florida| null|


In [10]:
df_sp.count()


4582

In [11]:
tweet_df2 = df_sp.select('tweet_id','tweet','location').distinct()

In [12]:
tweet_df2.show()

+--------+--------------------+-------------+
|tweet_id|               tweet|     location|
+--------+--------------------+-------------+
|     317|RT @JoeBiden: Fol...|     Michigan|
|     400|RT @charles_gaba:...|         Ohio|
|     546|I will be, Mr. Pr...|Massachusetts|
|     638|RT @rmslim: Frien...|     New York|
|     948|RT @KyleLacio: Mc...|   New Jersey|
|    1000|RT @DavidCaltabia...|      Arizona|
|    1308|RT @PowerDNS_Bert...|Massachusetts|
|    2133|@eugenegu Why did...|     New York|
|    2375|RT @HighImpactFli...|     Michigan|
|    2742|RT @NABTU: When w...|     New York|
|    4122|Johnson &amp; Joh...|     New York|
|    4245|@RealDoctorMike I...|   New Jersey|
|    5052|RT @malaykaridwan...|  Puerto Rico|
|    5061|RT @FowlerSarah: ...|  Mississippi|
|    5152|RT @RepAdamSchiff...|      Florida|
|    5193|RT @samstein: NEW...|        Texas|
|      15|RT @BioNTech_Grou...|     New York|
|     124|RT @harrisonjaime...|        Texas|
|     263|RT @JoeBiden: Fol...|   

In [13]:
tweet_df2.count()

4582

In [14]:
tweet_df4 = tweet_df2.select('tweet','location')

In [15]:
pand_df = tweet_df4.toPandas()


In [16]:
pand_df

,tweet,location
0,"RT @JoeBiden: Folks, I just received the secon...",Michigan
1,RT @charles_gaba: Holy shit. They’re holding v...,Ohio
2,"I will be, Mr. President. Need to know when th...",Massachusetts
3,"RT @rmslim: Friends, all of whom are physician...",New York
4,RT @KyleLacio: McMaster is skipping the grocer...,New Jersey
...,...,...
4577,.@MEPublicHealth @newscentermaine Retired nurs...,Maine
4578,RT @NationHahn: I loved hearing about @NCHealt...,North Carolina
4579,Can Flu Vaccine Increase COVID Risk? • Childre...,Texas
4580,RT @yattypat: Mexican doctor hospitalized afte...,Texas


In [17]:

host_name = os.environ.get('host_name')
user = "group2"
passwd = os.environ.get('aws_password')
db="group2db"
# connect to MySQL
con = pymysql.connect(host=host_name,user=user,passwd=passwd,db=db)
cursor = con.cursor()
# #### deleting a table in mysql
delete_table="""DROP TABLE IF EXISTS `tweets_locations`;"""
cursor.execute(delete_table)
# #

0

In [18]:

host_name = os.environ.get('host_name')
user = "group2"
passwd = os.environ.get('aws_password')
db="group2db"
# connect to MySQL
con = pymysql.connect(host=host_name,user=user,passwd=passwd,db=db)
cursor = con.cursor()
# #### creating a table for sending dataframe into mysql
create_table="""CREATE TABLE tweets_locations(tweet_id INT(11) not null auto_increment primary key,  tweet TEXT,
               location VARCHAR(100));"""
cursor.execute(create_table)
# #



0

In [19]:
cols = "`,`".join([str(i) for i in pand_df.columns.tolist()])

# Insert DataFrame records one by one.
for i,row in pand_df.iterrows():
    sql = "INSERT INTO `tweets_locations` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))

    # the connection is not autocommitted by default, so we must commit to save our changes
    con.commit()